In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create networkmodel = Network('MyNetwork')# Block 1: nodesnode1 = Delay(model, 'SlowDelay')node2 = Queue(model, 'FCFSQueue', SchedStrategy.FCFS)node3 = Queue(model, 'SIROQueue', SchedStrategy.SIRO)  # Service In Random Ordernode4 = Queue(model, 'PSQueue', SchedStrategy.PS)      # Processor Sharingnode5 = Queue(model, 'HOLQueue', SchedStrategy.HOL)    # Head-of-Line prioritynode6 = Delay(model, 'FastDelay')

In [3]:
# Block 2: classes with priorities
# Priority: 0=low, 1=high - Class2 has higher priority
jobclass1 = ClosedClass(model, 'Class1', 18, node1, 0)  # Low priority
jobclass2 = ClosedClass(model, 'Class2', 18, node1, 1)  # High priority
jobclass3 = ClosedClass(model, 'Class3', 18, node1, 0)  # Low priority

In [4]:
# Set service times - identical at delay stations for fair comparison
# SlowDelay: all classes same service time
node1.set_service(jobclass1, Exp.fitMean(10.0))
node1.set_service(jobclass2, Exp.fitMean(10.0))
node1.set_service(jobclass3, Exp.fitMean(10.0))

# FCFSQueue: different service times per class
node2.set_service(jobclass1, Exp.fitMean(0.3))
node2.set_service(jobclass2, Exp.fitMean(0.5))
node2.set_service(jobclass3, Exp.fitMean(0.6))

# SIROQueue: different service times per class
node3.set_service(jobclass1, Exp.fitMean(1.1))
node3.set_service(jobclass2, Exp.fitMean(1.3))
node3.set_service(jobclass3, Exp.fitMean(1.5))

# PSQueue: different service times per class
node4.set_service(jobclass1, Exp.fitMean(1.0))
node4.set_service(jobclass2, Exp.fitMean(1.1))
node4.set_service(jobclass3, Exp.fitMean(1.9))

# HOLQueue: different service times per class - priority will matter here
node5.set_service(jobclass1, Exp.fitMean(2.5))
node5.set_service(jobclass2, Exp.fitMean(1.9))
node5.set_service(jobclass3, Exp.fitMean(4.3))

# FastDelay: all classes same service time
node6.set_service(jobclass1, Exp.fitMean(1.0))
node6.set_service(jobclass2, Exp.fitMean(1.0))
node6.set_service(jobclass3, Exp.fitMean(1.0))

In [5]:
# Create cyclic routing: SlowDelay -> FCFS -> SIRO -> PS -> HOL -> FastDelay -> SlowDelay
P = model.init_routing_matrix()

# All classes follow the same cyclic route
for jobclass in [jobclass1, jobclass2, jobclass3]:
    P.set(jobclass, jobclass, node1, node2, 1.0)  # SlowDelay -> FCFSQueue
    P.set(jobclass, jobclass, node2, node3, 1.0)  # FCFSQueue -> SIROQueue
    P.set(jobclass, jobclass, node3, node4, 1.0)  # SIROQueue -> PSQueue
    P.set(jobclass, jobclass, node4, node5, 1.0)  # PSQueue -> HOLQueue
    P.set(jobclass, jobclass, node5, node6, 1.0)  # HOLQueue -> FastDelay
    P.set(jobclass, jobclass, node6, node1, 1.0)  # FastDelay -> SlowDelay

model.link(P)

In [ ]:
# SolverMVA with seed=23000 to match JAR test
solver = SolverMVA(model, seed=23000, verbose=VerboseLevel.SILENT)
avg_table = solver.get_avg_table()

In [ ]:
# SolverSSA with seed=23000, samples=10000 to match JAR test
solver = SolverSSA(model, seed=23000, samples=10000, verbose=VerboseLevel.SILENT)
avg_table = solver.get_avg_table()